In [1]:
import torch
print(torch.__version__)
torch.cuda.is_available()

2.4.1+cu121


True

In [2]:
!nvidia-smi

Sat Dec 14 15:19:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        On  |   00000000:98:00.0 Off |                  N/A |
| 50%   43C    P2            187W /  320W |    1204MiB /  20480MiB |     54%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
for i in range(100,200):
    if 55760%i == 0:
        print(i)

136
164
170
